# Hidden Discrete Markov Models

### Importing Required Libraries

In [2]:
from typing import Tuple

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
# Parameters for our distribution p(observation|latent). These will be long and
# randomized
np.random.seed(2)
obs_dim = 3
GAUSS_MEANS = [np.array([1.0, -1.0, 0.2]),
               np.array([0.6, 1.1, 1.0]),
               np.array([1.3, 1.0, 1.2])]
GAUSS_COVS = [np.eye(obs_dim) / 4]*3
PI = np.array([0.7, 0.2, 0.1])
A_MAT = np.array([[0.9, 0.0, 0.1], [0.0, 0.9, 0.1], [0.1, 0.3, 0.6]])

def generate_latent_and_observations(n_observations: int) -> Tuple[np.ndarray, np.ndarray]:
    """Generate a sample set of observations along with the true latents.

    Args:
        n_observations: Number of observations to generate.

    Returns:
        Sequence of latent states and observations.
    """
    # Placeholders.
    latents = np.zeros((n_observations + 1, len(PI)))
    observations = np.zeros((n_observations, len(GAUSS_MEANS[0])))

    # Start with the initial latent state
    latents[0, np.random.choice(np.arange(len(PI)), p=PI)] = 1

    # Fill out the rest.
    for i in range(n_observations):
        latents[i+1, np.random.choice(np.arange(len(PI)), p=A_MAT[np.argmax(latents[i])])] = 1
        cur_state = np.argmax(latents[i+1])
        observations[i] = np.random.multivariate_normal(GAUSS_MEANS[cur_state], GAUSS_COVS[cur_state])

    return latents, observations

def observation_probability(latent_index: int, observation: np.ndarray) -> float:
    """Given an observation and corresponding latent state, evaluate the likelihood.

    Args:
        latent_index: Index of latent state encoding.
        observation: Observation at current time step.

    Returns:
        Likelihood p(observation|latent).
    """
    # Let's keep things somewhat 'simple' by making our distribution a sum of Gaussians
    return stats.multivariate_normal.pdf(observation, mean = GAUSS_MEANS[latent_index], cov = GAUSS_COVS[latent_index])

# Generate the data for our study.
true_latents, observations = generate_latent_and_observations(1000)

## Emotion Study

Your friend in the sociology department is studying the effects of social media habits on emotions. They’ve run an experiment placing FMRI sensors on the brains of several volunteers as they scroll through the internet. The volunteers can be in one of three states: happy, angry, or neutral. They’ve used the following one-hot encoding:

$$
\mathbf{z}_{\text{happy}} = \begin{bmatrix}1\\0\\0\end{bmatrix}, \quad
\mathbf{z}_{\text{angry}} = \begin{bmatrix}0\\1\\0\end{bmatrix}, \quad
\mathbf{z}_{\text{neutral}} = \begin{bmatrix}0\\0\\1\end{bmatrix}.
$$

Given that encoding, they also have a very good model for the probability of observing a specific FMRI scan $ \mathbf{x}_t $ given the state $ \mathbf{z}_t $. They have already done the favor of uploading it to your notebook as the function `observation_probability`. They also have a time series $ \{\mathbf{x}_1, \dots, \mathbf{x}_T\} $ for a volunteer that spends a lot of time on social media. For this volunteer, they know that the transition matrix looks like:

$$
A = 
\begin{bmatrix}
0.9 & 0.0 & 0.1 \\
0.0 & 0.9 & 0.1 \\
0.1 & 0.3 & 0.6
\end{bmatrix},
$$

and having observed the volunteer enter they have a pretty good guess at the initial state:

$$
\boldsymbol{\pi} = 
\begin{bmatrix}
0.7 \\
0.2 \\
0.1
\end{bmatrix}.
$$

They would like to know:

- **What is the most likely emotional state at each time step** (i.e. max of $p(z_t | \mathbf{x}_{1:t})$ or $p(z_t | \mathbf{x}_{1:T})$)?
- **What is the most likely sequence of emotional states expressed by the volunteer?**


### Part I: Implementing the Viterbi Algorithm for a discrete Hidden Markov Model.

To answer your friend’s questions, you will need to solve for the transition matrix governing the discrete HMM and calculate the most likely series of latent states. This will require:

1. Implementing the calculation for `α̂(z_t)`
2. Implementing the calculation for `β̂(z_t)`
3. Implementing the calculation for `p(z_t | x₁:t)`, `p(z_t | x₁:T)`, `p(z₁:T | x₁:T)`
4. Implementing the Viterbi algorithm.